初始化包

In [20]:
from selenium import webdriver
# 使用键盘按键方法要导入下面的类
from selenium.webdriver.common.keys import Keys
import time
import csv
import os
from dotenv import load_dotenv

# 从.env文件中导入账号(ACCOUNT)和密码(PASSWORD)
dotenv_path = "../../.env"
load_dotenv(dotenv_path)
ACCOUNT = os.getenv("HNJD_ACCOUNT")
PASSWORD = os.getenv("HNJD_PASSWORD")

print("import success")

import success


进入主页

In [21]:
driver = webdriver.Chrome()

driver.get(
    "http://authserver.hnjdzy.net/authserver/login?service=http%3A%2F%2Fehall.hnjdzy.net%2Flogin%3Fservice%3Dhttp%3A%2F%2Fehall.hnjdzy.net%2Fnew%2Findex.html"
)

driver.implicitly_wait(3)

username = driver.find_element_by_name("username")
password = driver.find_element_by_id("password")
username.send_keys(ACCOUNT)
password.send_keys(PASSWORD)
submit = driver.find_element_by_class_name("auth_login_btn")
submit.click()



In [22]:
# 获取当前窗口句柄
all_handler = driver.window_handles
driver.switch_to_window(all_handler[1])

C:\Users\YuMin\AppData\Local\Temp\ipykernel_5044\3949258735.py:3: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(all_handler[1])


In [23]:
import re
def sanitize_filename(filename):
    # 移除非法字符
    filename = re.sub(r'[\/:*?"<>|]', '_', filename)
    # 限制文件名长度
    return filename[:255]

def get_file_extension(url):
    # 去掉URL中的查询参数
    clean_url = url.split('?')[0]
    # 获取文件扩展名
    file_extension = clean_url.split('.')[-1]
    return file_extension

print("done")

done


核心处理逻辑

In [45]:
score_type_dict = {
    "参加学生活动": "学生",
    "其他加分": "加分",
    "导师制活动": "导师制",
    "担任干部": "干部"
}

# 遍历证据csv文件，上传证据并逐个提交文件
with open('proofs.csv', 'r', newline='', encoding='utf-8') as csvfile:
    # 创建CSV读取对象
    reader = csv.DictReader(csvfile)
    # 遍历每一行数据
    for row in reader:
        # 点击“新增”按钮
        newFjf = driver.find_element_by_id("newFjf")
        newFjf.click()

        # 点击“附加分名称”，找到附加分名称输入框，输入对应项目内容，点击对应项目
        pullbox = driver.find_element_by_css_selector('label[title="附加分名称"] + div div[id^="dropdownlistArrowjqxWidget"]')
        pullbox.click()
        searchBoxes = driver.find_elements_by_css_selector('div[id^="filterinnerListBoxjqxWidget"]')
        searchBox = searchBoxes[0].find_element_by_css_selector('input')
        # 自定义位置 -> 分数类型
        searchBox.send_keys(score_type_dict[row["分数类型"]])
        searchBox.send_keys(Keys.ENTER)
        time.sleep(.5)
        search_box_items = driver.find_elements_by_css_selector('div[id^="listBoxContentinnerListBoxjqxWidget"]')
        search_box_item = search_box_items[0].find_element_by_css_selector('div:nth-child(1) > div:nth-child(1)')
        search_box_item.click()

        # 点击“得分”下拉框，输入“3”，点击对应项目
        if row["分数类型"] != "其他加分":
            score_box = driver.find_element_by_css_selector('label[title="得分"] + div div[id^="dropdownlistArrowjqxWidget"]')
            score_box.click()
            score_search = searchBoxes[1].find_element_by_css_selector('input')
            # 自定义位置 -> 分数
            score_search.send_keys(row["分数"])
            score_s_result = search_box_items[1].find_element_by_css_selector('div:nth-child(1) > div:nth-child(1)')
            time.sleep(.5)
            if score_s_result.text != '':
                score_s_result.click()
        else:
            score_box = driver.find_element_by_css_selector('label[title="得分"] + div input')
            score_box.send_keys(row["分数"])


        # 点击“申请说明”框，输入预期结果
        description_box = driver.find_element_by_class_name("bh-txt-input__txtarea")
        description_box.send_keys(row["标题"] + " " + row["时间"])

        ROOT_PATH = "E:/python/practical_py_data_utils/hnjd/social_credit/"

        # 找到文件控件，直接根据路径上传文件
        file_input = driver.find_element_by_name("bhFile")
        # 获取第一个图片的路径并上传
        photo_names = row['图片名称'].split(', ')
        image_path = ROOT_PATH + "res/" + photo_names[0]
        file_input.send_keys(image_path)
        # 等待5秒
        time.sleep(5)
        # 点击“提交”按钮
        buttons = driver.find_elements_by_class_name("bh-pull-right")
        submit_btn = buttons[1]
        submit_btn.click()

        # faild file 
print("done")

ElementClickInterceptedException: Message: element click intercepted: Element <button type="button" id="newFjf" class="bh-btn bh-btn-primary">...</button> is not clickable at point (193, 374). Other element would receive the click: <div class="jqx-window-modal" style="opacity: 0.3; display: block; position: absolute; top: 0px; left: 0px; width: 100%; height: 2434px; z-index: 18000;"></div>
  (Session info: chrome=125.0.6422.142)


In [13]:
print(driver.title)

综合测评
